In [20]:
# 데이터 불러오기

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

In [23]:
# 간단한 CNN 구축

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 14 * 14, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 32 * 14 * 14)
        x = self.fc1(x)
        return x

In [ ]:
from tqdm import tqdm

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3):  # 3 epochs만 돌림 (시간 절약)
    running_loss = 0.0
    progress_bar = tqdm(trainloader, desc=f'Epoch {epoch+1}')
    
    for images, labels in progress_bar:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        # 현재까지의 평균 loss를 progress bar에 표시
        progress_bar.set_postfix({'loss': f'{running_loss/len(trainloader):.3f}'})

print('학습 완료!')

In [ ]:
# 모델이 어떤 디바이스에서 실행되고 있는지 확인
print(f"Model device: {next(model.parameters()).device}")

# 입력 데이터의 디바이스 확인
for images, labels in trainloader:
    print(f"Input data device: {images.device}")
    break

In [ ]:
# 결과확인

import matplotlib.pyplot as plt
import numpy as np

# 예제 이미지 출력
dataiter = iter(trainloader)
images, labels = next(dataiter)
plt.imshow(images[0].numpy().squeeze(), cmap='gray')
plt.show()

In [ ]:
# 테스트 샘플 예측

outputs = model(images)
_, predicted = torch.max(outputs, 1)
print(f"예측된 숫자: {predicted[0].item()}")

# 성능 개선 NVIDIA CUDA GPU

In [26]:
from torchvision import datasets, transforms

# MNIST 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST의 평균과 표준편차
])

# 데이터셋 다운로드
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 이후 코드는 동일
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
# MPS 사용 가능 여부 확인 (Mac용)
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS 사용 가능")
elif torch.cuda.is_available():    # GPU 사용 가능 여부 확인
    device = torch.device("cuda")
    print("CUDA GPU 사용 가능")
else:
    device = torch.device("cpu")
    print("CPU만 사용 가능")

print(f"Using device: {device}")

# 모델을 해당 디바이스로 이동
model = SimpleCNN().to(device)

# 데이터 로더 설정
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

from tqdm import tqdm

for epoch in range(3):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(trainloader, desc=f'Epoch {epoch+1}')
    
    for images, labels in progress_bar:
        # 데이터를 해당 디바이스로 이동
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        # 현재까지의 평균 loss를 progress bar에 표시
        progress_bar.set_postfix({'loss': f'{running_loss/len(trainloader):.3f}'})

print('학습 완료!')

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델을 GPU로 이동
model = SimpleCNN().to(device)

# 데이터 로더 설정 수정
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(3):
    for images, labels in trainloader:
        # 데이터를 GPU로 이동
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
# 모델이 어떤 디바이스에서 실행되고 있는지 확인
print(f"Model device: {next(model.parameters()).device}")

# 입력 데이터의 디바이스 확인
for images, labels in trainloader:
    print(f"Input data device: {images.device}")
    break

# 성능개선 by Mac

In [ ]:
# MPS 디바이스 사용 가능 여부 확인
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS 사용 가능")
else:
    device = torch.device("cpu")
    print("MPS 사용 불가능 - CPU 사용")

print(f"Using device: {device}")

# 모델을 MPS로 이동
model = SimpleCNN().to(device)

In [ ]:
# 모델을 MPS로 이동
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 진행 상황을 보기 위해 tqdm 추가
from tqdm import tqdm

for epoch in range(3):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(trainloader, desc=f'Epoch {epoch+1}')
    
    for images, labels in progress_bar:
        # 데이터를 MPS로 이동
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        progress_bar.set_postfix({'loss': f'{running_loss/len(trainloader):.3f}'})

print('학습 완료!')

In [ ]:
# 결과확인

import matplotlib.pyplot as plt
import numpy as np

# 예제 이미지 출력
dataiter = iter(trainloader)
images, labels = next(dataiter)
plt.imshow(images[0].numpy().squeeze(), cmap='gray')
plt.show()

In [ ]:
# 테스트 샘플 예측
images = images.to(device)  # 이미지를 MPS로 이동
outputs = model(images)
_, predicted = torch.max(outputs, 1)
print(f"예측된 숫자: {predicted[0].item()}")